### Evaluation of UQ - Accuracy Rejection Curves

In [1]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from math import log
import numpy as np

In [2]:
# Uncertainty from Model 1, Model 1+2, Model 1+2+3, 
# instead of Model 1, Model 2, Model 3, 
# because n+1 Model always belong to Model n
# in an additive manner

# catboost delivers already laplace_smoothed_proba_predictions


def uncertainty(estimators, test_data):
    # function expects a list of estimators and test data 
    # and returns aleatoric, epistemic and total uncertainty
    # for each prediction
    

    labels = [0,1] # labels are 0 or 1 for this task 
    num_trees = len(estimators) # number of trees
    num_samples =  len(test_data) # number of samples
    predictions = [] # init predictions
    
    
    # calculate laplace corrected predictions
    # for each estimator and test sample
    
    for m in estimators:
        leave_id = m.apply(x_test)
        laplace_corrected_prediction = []
        
        for frequencies in [el[0] for el in m.tree_.value[leave_id]]:
            first_class_proba = (frequencies[0]+1)/(frequencies[0]+frequencies[1]+2)
            second_class_proba = (frequencies[1]+1)/(frequencies[0]+frequencies[1]+2)                  
            laplace_corrected_prediction.append([first_class_proba,second_class_proba])
            
        predictions.append(laplace_corrected_prediction)
        
    # init uncertaintys
    total = []
    aleatoric = []
    epistemic = []
    
    # ALEATORIC UNCERTAINTY
    for i in range(num_samples):
        result = 0
        for m in range(num_trees):
            for y in labels:
                p = predictions[m][i][y]
                result = result + (p * log(p,2))
        
        final = -(1/num_trees) * result

        aleatoric.append(final) 

    p_sum = 0
    for i in range(num_samples):
        result = 0
        for y in labels:
            p_sum = 0
            for m in range(num_trees):
                
                p = predictions[m][i][y]
                p_sum = p + p_sum
            
            p_avg_sum = ((1/num_trees)* p_sum)
            p_logged_avg_sum = p_avg_sum*log(p_avg_sum,2)
            result = result + p_logged_avg_sum
            
        result = result *(-1)
        total.append(result)
        epistemic.append(result - aleatoric[i])
    
    return aleatoric, epistemic, total

In [3]:
def get_prob_matrix(model, x_test, n_estimators, laplace_smoothing, log=False):
    porb_matrix = [[[] for j in range(n_estimators)] for i in range(x_test.shape[0])]
    for etree in range(n_estimators):
        # populate the porb_matrix with the tree_prob
        tree_prob = model.estimators_[etree].predict_proba(x_test)
        if laplace_smoothing > 0:
            leaf_index_array = model.estimators_[etree].apply(x_test)
            for data_index, leaf_index in enumerate(leaf_index_array):
                leaf_values = model.estimators_[etree].tree_.value[leaf_index]
                leaf_samples = np.array(leaf_values).sum()
                for i,v in enumerate(leaf_values[0]):
                    tmp = (v + laplace_smoothing) / (leaf_samples + (len(leaf_values[0]) * laplace_smoothing))
                    # print(f">>>>>>>>>>>>>>> {tmp}  data_index {data_index} prob_index {i} v {v} leaf_samples {leaf_samples}")
                    tree_prob[data_index][i] = (v + laplace_smoothing) / (leaf_samples + (len(leaf_values[0]) * laplace_smoothing))
                # exit()

        for data_index, data_prob in enumerate(tree_prob):
            porb_matrix[data_index][etree] = list(data_prob)

        if log:
            print(f"----------------------------------------[{etree}]")
            print(f"class {model.estimators_[etree].predict(x_test)}  prob \n{tree_prob}")
    return porb_matrix

def uncertainty_estimate(probs): # three dimentianl array with d1 as datapoints, (d2) the rows as samples and (d3) the columns as probability for each class
    p = np.array(probs)
    entropy = -p*np.ma.log10(p)
    entropy = entropy.filled(0)
    a = np.sum(entropy, axis=1)
    a = np.sum(a, axis=1) / entropy.shape[1]
    p_m = np.mean(p, axis=1)
    total = -np.sum(p_m*np.ma.log10(p_m), axis=1)
    total = total.filled(0)
    e = total - a
    return total, e, a

#### SPECT

In [4]:
train = pd.read_csv('SPECT .train', delimiter=',', header=None)
test = pd.read_csv('SPECT .test', delimiter=',',header=None)
data = pd.concat([train,test]).reset_index(drop=True)
x = data.drop(0, axis=1)
y = data[0]

In [5]:
num_runs = 100
n_estimators = 50 # number of trees in the forest
test_size = 0.3
laplace_smoothing = 1
unc_method = "entropy"


ARC_eU_list, ARC_aU_list, ARC_random_list = [],[],[]


for i in range(100):
    test_size=0.3

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size,shuffle=True)
    x_train, x_test, y_train, y_test = x_train.copy(), x_test.copy(), y_train.copy(), y_test.copy()

    model = RandomForestClassifier(bootstrap=True,
                                        criterion='entropy',
                                        max_depth=10,
                                        n_estimators=n_estimators,
                                        random_state=None,
                                        verbose=0,
                                        warm_start=False)
    model.fit(x_train, y_train)

    preds_class = model.predict(x_test)
    print("Accuracy = {}".format(accuracy_score(y_test, preds_class)))

    porb_matrix = get_prob_matrix(model, x_test, n_estimators, laplace_smoothing, log=0)
    total_uncertainty, epistemic_uncertainty, aleatoric_uncertainty = uncertainty_estimate(np.array(porb_matrix))
    
    x_test['a']=aleatoric_uncertainty
    x_test['e']=epistemic_uncertainty

    num_rows = x_test.shape[0]
    discard_count = 0
    eU = x_test.sort_values('e', ascending=True)
    aU = x_test.sort_values('a', ascending=True)

    ARC_eU =  [preds_class]
    ARC_aU = [preds_class]
    ARC_random = [preds_class]

    i = 0.05
    while i < 1:
        discard_count = int (num_rows*i)
        eU_samples = eU.iloc[:-discard_count,0:22]
        aU_samples = aU.iloc[:-discard_count,0:22]
        random_samples = x_test.iloc[:,0:22].sample((num_rows-discard_count))
        preds_class_eU = model.predict(eU_samples)
        preds_class_aU = model.predict(aU_samples)
        preds_class_random = model.predict(random_samples)
        accuracy_eu = accuracy_score(y_test[eU_samples.index], preds_class_eU)
        accuracy_au = accuracy_score(y_test[aU_samples.index], preds_class_aU)
        accuracy_random = accuracy_score(y_test[random_samples.index], preds_class_random)
        ARC_eU.append(accuracy_eu)
        ARC_aU.append(accuracy_au)
        ARC_random.append(accuracy_random)
        i+=0.05
    
    ARC_eU_list.append(ARC_eU)
    ARC_aU_list.append(ARC_aU)
    ARC_random_list.append(ARC_random)

Accuracy = 0.7901234567901234
Accuracy = 0.7777777777777778
Accuracy = 0.8024691358024691
Accuracy = 0.8271604938271605
Accuracy = 0.8518518518518519
Accuracy = 0.8024691358024691
Accuracy = 0.7530864197530864
Accuracy = 0.7654320987654321
Accuracy = 0.9012345679012346
Accuracy = 0.8271604938271605
Accuracy = 0.8765432098765432
Accuracy = 0.8024691358024691
Accuracy = 0.8395061728395061
Accuracy = 0.8395061728395061
Accuracy = 0.8395061728395061
Accuracy = 0.7654320987654321
Accuracy = 0.8518518518518519
Accuracy = 0.8395061728395061
Accuracy = 0.7777777777777778
Accuracy = 0.8271604938271605
Accuracy = 0.7901234567901234
Accuracy = 0.8395061728395061
Accuracy = 0.8641975308641975
Accuracy = 0.8518518518518519
Accuracy = 0.8395061728395061
Accuracy = 0.8395061728395061
Accuracy = 0.8271604938271605
Accuracy = 0.8024691358024691
Accuracy = 0.8641975308641975
Accuracy = 0.7654320987654321
Accuracy = 0.7901234567901234
Accuracy = 0.8024691358024691
Accuracy = 0.7777777777777778
Accuracy =

In [6]:
ARC_eU_mean, ARC_aU_mean, ARC_random_mean = [],[],[]
for i in range(len(ARC_eU_list[0])):
    ARC_eU_mean.append(np.array([el[i] for el in ARC_eU_list]).mean())
    ARC_aU_mean.append(np.array([el[i] for el in ARC_aU_list]).mean())
    ARC_random_mean.append(np.array([el[i] for el in ARC_random_list]).mean())

In [7]:
print('Reject   E    A    R ')
i = 0.05
for e,a,r in zip(ARC_eU_mean,ARC_aU_mean,ARC_random_mean):
    print(f'{i:.2f}:  {e:.2f} {a:.2f} {r:.2f} ')
    i +=0.05

Reject   E    A    R 
0.05:  0.83 0.83 0.83 
0.10:  0.82 0.83 0.82 
0.15:  0.83 0.84 0.82 
0.20:  0.83 0.85 0.82 
0.25:  0.83 0.88 0.82 
0.30:  0.84 0.90 0.82 
0.35:  0.84 0.92 0.82 
0.40:  0.85 0.93 0.82 
0.45:  0.86 0.93 0.82 
0.50:  0.86 0.94 0.82 
0.55:  0.87 0.94 0.82 
0.60:  0.88 0.94 0.82 
0.65:  0.88 0.95 0.82 
0.70:  0.88 0.95 0.82 
0.75:  0.89 0.97 0.82 
0.80:  0.89 0.97 0.82 
0.85:  0.89 0.98 0.80 
0.90:  0.91 0.99 0.82 
0.95:  0.93 1.00 0.82 
1.00:  0.98 1.00 0.84 
